In [12]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
input_files = [
  'edgar_allan_poe.txt',
  'robert_frost.txt',
]

In [15]:
!head edgar_allan_poe.txt

LO! Death hath rear'd himself a throne
In a strange city, all alone,
Far down within the dim west
Where the good, and the bad, and the worst, and the best,
Have gone to their eternal rest.
 
There shrines, and palaces, and towers
Are not like any thing of ours
Oh no! O no! ours never loom
To heaven with that ungodly gloom!


In [16]:
!head robert_frost.txt

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there


In [17]:
# collect data into lists
input_texts = []
labels = []

for label, f in enumerate(input_files):
  print(f"{f} corresponds to label {label}")

  for line in open(f):
    line = line.rstrip().lower()
    if line:
      # remove punctuation
      line = line.translate(str.maketrans('', '', string.punctuation))

      input_texts.append(line)
      labels.append(label)

edgar_allan_poe.txt corresponds to label 0
robert_frost.txt corresponds to label 1


In [18]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [19]:
len(Ytrain), len(Ytest)

(1615, 539)

In [20]:
train_text[:5]

['making allowance making due allowance',
 'for god man or beasts sake',
 'in slightly sinking the dull tide',
 'they have best right to be heard in this place',
 'summoning spirits isnt button button']

In [21]:
Ytrain[:5]

[1, 1, 0, 1, 1]

In [22]:
idx = 1
word2idx = {'<unk>': 0}

In [23]:
# populate word2idx
for text in train_text:
    tokens = text.split()
    for token in tokens:
      if token not in word2idx:
        word2idx[token] = idx
        idx += 1

In [24]:
word2idx

{'<unk>': 0,
 'making': 1,
 'allowance': 2,
 'due': 3,
 'for': 4,
 'god': 5,
 'man': 6,
 'or': 7,
 'beasts': 8,
 'sake': 9,
 'in': 10,
 'slightly': 11,
 'sinking': 12,
 'the': 13,
 'dull': 14,
 'tide': 15,
 'they': 16,
 'have': 17,
 'best': 18,
 'right': 19,
 'to': 20,
 'be': 21,
 'heard': 22,
 'this': 23,
 'place': 24,
 'summoning': 25,
 'spirits': 26,
 'isnt': 27,
 'button': 28,
 'only': 29,
 'i': 30,
 'happen': 31,
 'prefer': 32,
 'live': 33,
 'and': 34,
 'both': 35,
 'of': 36,
 'our': 37,
 'united': 38,
 'strengths': 39,
 'do': 40,
 'it': 41,
 'drawn': 42,
 'into': 43,
 'town': 44,
 'about': 45,
 'cellar': 46,
 'hole': 47,
 'builder': 48,
 'building': 49,
 'little': 50,
 'house': 51,
 'but': 52,
 'theres': 53,
 'a': 54,
 'dite': 55,
 'too': 56,
 'many': 57,
 'them': 58,
 'comfort': 59,
 'she': 60,
 'has': 61,
 'seen': 62,
 'that': 63,
 'tears': 64,
 'are': 65,
 'not': 66,
 'dry': 67,
 'on': 68,
 'by': 69,
 'picking': 70,
 'faded': 71,
 'blue': 72,
 'joy': 73,
 'breathing': 74,
 'is

In [25]:
len(word2idx)

2528

In [26]:
# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
  line_as_int = [word2idx.get(token, 0) for token in tokens]
  test_text_int.append(line_as_int)

In [27]:
train_text_int[100:105]

[[10, 391, 36, 392, 393],
 [10, 13, 394, 36, 13, 395, 396],
 [352, 65, 97, 397, 170, 13, 6, 398, 399],
 [7, 126, 400, 68, 401, 402, 66, 20, 324],
 [233, 131, 82, 297, 17, 403, 404, 151]]

In [28]:
# initialize A and pi matrices - for both classes
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)

In [29]:
# compute counts for A and pi
def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        # it's the first word in a sentence
        pi[idx] += 1
      else:
        # the last word exists, so count a transition
        A[last_idx, idx] += 1

      # update last idx
      last_idx = idx


compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

In [30]:
# normalize A and pi so they are valid probability matrices
# convince yourself that this is equivalent to the formulas shown before
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [31]:
# log A and pi since we don't need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [32]:
# compute priors
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.339938080495356, 0.660061919504644)

In [33]:
# build a classifier
class Classifier:
  def __init__(self, logAs, logpis, logpriors):
    self.logAs = logAs
    self.logpis = logpis
    self.logpriors = logpriors
    self.K = len(logpriors) # number of classes

  def _compute_log_likelihood(self, input_, class_):
    logA = self.logAs[class_]
    logpi = self.logpis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        # it's the first token
        logprob += logpi[idx]
      else:
        logprob += logA[last_idx, idx]

      # update last_idx
      last_idx = idx

    return logprob

  def predict(self, inputs):
    predictions = np.zeros(len(inputs))
    for i, input_ in enumerate(inputs):
      posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
             for c in range(self.K)]
      pred = np.argmax(posteriors)
      predictions[i] = pred
    return predictions

In [34]:
# each array must be in order since classes are assumed to index these lists
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [35]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.9962848297213622


In [36]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")

Test acc: 0.8423005565862709


In [37]:
from sklearn.metrics import confusion_matrix, f1_score

# read about F-score: https://en.wikipedia.org/wiki/F-score

In [38]:
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[ 543,    6],
       [   0, 1066]])

In [39]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[ 92,  77],
       [  8, 362]])

In [40]:
f1_score(Ytrain, Ptrain)

0.9971936389148737

In [41]:
f1_score(Ytest, Ptest)

0.8949320148331273